In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

upload 'shhs1-dataset-0.20.0 (1).csv' and 'shhs-data-dictionary-0.20.0-variables.csv'

In [2]:
shhs1 = pd.read_csv('shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)
var_dict = pd.read_csv('shhs-data-dictionary-0.20.0-variables.csv')

<ipython-input-2-ac23a95ac129>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  shhs1 = pd.read_csv('shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)
<ipython-input-2-ac23a95ac129>:1: DtypeWarning: Columns (1214) have mixed types. Specify dtype option on import or set low_memory=False.
  shhs1 = pd.read_csv('shhs1-dataset-0.20.0 (1).csv', error_bad_lines=False)


In [3]:
shhs1.shape

(5804, 1271)

In [4]:
var_dict.shape

(1896, 11)

# Data Processing

In [5]:
# remove sleep monitoring columns from independent variables (unattinable for predicting)
sleep_monitoring_col = var_dict[var_dict['folder'].str.contains(r'sleep monitoring', case=False, na=False)]['id']

In [6]:
sleep_monitoring_col

633        ahiov50
634     headbox_id
635        hrov150
636        hrund30
637     monitor_id
           ...    
1721      timest1p
1722       timest2
1723      timest2p
1724      timest34
1725          waso
Name: id, Length: 1093, dtype: object

In [7]:
# drop target variables, pptidr (has numerical and categorical values in one column), and sleep monitoring columns in independent variable
x = shhs1.drop(columns=['ahi_c0h3a', 'pptidr'])
for col in sleep_monitoring_col:
    if col in x.columns:
        x = x.drop(columns=col)

y = shhs1['ahi_c0h3a']

In [8]:
x

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7,1
1,200002,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,NaN,9,1
2,200003,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9,1
3,200004,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6,1
4,200005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,NaN,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800,5835,-854.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8,1
5800,205801,5836,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6,1
5801,205802,5837,-768.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7,1
5802,205803,5838,-755.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6,1


In [9]:
# normalize x
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

imputer = KNNImputer(n_neighbors=5)  # You can change the number of neighbors if needed
x_imputed_scaled = imputer.fit_transform(x_scaled)

# reverse scaling
x_imputed = scaler.inverse_transform(x_imputed_scaled)

# Convert the result back to DataFrame
x_imputed = pd.DataFrame(x_imputed, columns=x.columns)

problem with knn: how many neighbors are the best?
NaNs categorical variables will be imputed, but the result might not be intepretable

In [10]:
x_imputed

,nsrrid,pptid,ecgdate,lvh3_1,lvh3_3,st4_1_3,st5_1_3,lvhst,mob1,part2deg,...,educat,date02,date25,weight,waist,height,weight20,lang15,age_category_s1,visitnumber
0,200001.0,1.0,-748.2,0.4,0.6,0.2,2.000000e-01,0.2,0.0,0.0,...,3.0,-48.0,0.0,65.0,86.0,178.0,69.0,1.0,7.0,1.0
1,200002.0,2.0,-75.8,0.2,0.2,0.0,2.000000e-01,0.0,0.0,0.0,...,2.0,-111.0,-37.0,93.0,107.0,168.0,93.0,1.0,9.0,1.0
2,200003.0,3.0,-1071.8,0.2,0.0,0.0,4.000000e-01,0.0,0.0,0.0,...,3.0,-7.0,0.0,51.0,82.0,145.0,50.7,1.0,9.0,1.0
3,200004.0,4.0,-728.6,0.0,0.0,0.0,2.000000e-01,0.0,0.0,0.0,...,3.0,-24.0,0.0,64.0,85.0,180.0,65.4,1.0,6.0,1.0
4,200005.0,5.0,-241.8,0.2,0.0,0.0,2.775558e-17,0.0,0.0,0.0,...,2.0,-89.0,-33.0,56.0,76.0,155.0,56.0,1.0,8.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5799,205800.0,5835.0,-854.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-168.0,1.0,70.5,99.0,166.0,70.8,1.0,8.0,1.0
5800,205801.0,5836.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,3.0,-95.0,1.0,83.6,99.0,176.0,82.8,1.0,6.0,1.0
5801,205802.0,5837.0,-768.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,4.0,-83.0,1.0,75.0,91.0,175.0,74.2,1.0,7.0,1.0
5802,205803.0,5838.0,-755.0,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,...,2.0,-89.0,0.0,76.8,93.0,176.0,78.6,1.0,6.0,1.0


# Feature selection

## Decision Tree


In [11]:
tree = DecisionTreeRegressor()
tree.fit(x_imputed, y)

DecisionTreeRegressor()

In [12]:
importances = tree.feature_importances_
decision_tree_features = np.array(x_imputed.columns)[importances > 0]
decision_tree_dataset = x_imputed[decision_tree_features]
decision_tree_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']

<ipython-input-12-dbe2af9a44f8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decision_tree_dataset['ahi_c0h3a'] = shhs1['ahi_c0h3a']


In [13]:
decision_tree_dataset.to_csv('decision_tree_dataset.csv', index=False)

## Random Forest

In [14]:
forest = RandomForestRegressor(n_estimators=100)  # You can change the number of trees if needed
forest.fit(x_imputed, y)

RandomForestRegressor()

In [15]:
importances = forest.feature_importances_

In [16]:
# Get the indices of the features sorted by importance
sorted_indices = np.argsort(importances)[::-1]

# Print the feature rankings
for i, idx in enumerate(sorted_indices):
    print(f"{i+1}. Feature {idx} ({x_imputed.columns[idx]}): {importances[idx]}")

1. Feature 71 (neck20): 0.13901447473598227
2. Feature 282 (bmi_s1): 0.05598492021523472
3. Feature 287 (waist): 0.03279284279934924
4. Feature 289 (weight20): 0.024703605877309215
5. Feature 262 (age_s1): 0.023753487960845768
6. Feature 2 (ecgdate): 0.016165424922965972
7. Feature 213 (yrssnr02): 0.014293007166441525
8. Feature 47 (trig): 0.013991311440739427
9. Feature 45 (chol): 0.013982002929951822
10. Feature 50 (aai): 0.013627509218939
11. Feature 261 (mcs_s1): 0.013518693267656303
12. Feature 48 (fev1): 0.013143231979195194
13. Feature 286 (weight): 0.013047322451647752
14. Feature 284 (date02): 0.012599470399050208
15. Feature 95 (yrsns15): 0.012350931535131055
16. Feature 43 (diasbp): 0.01173375757934303
17. Feature 288 (height): 0.011658019661169226
18. Feature 28 (qrs): 0.011639408462458763
19. Feature 46 (hdl): 0.011535446611425247
20. Feature 97 (cigday15): 0.011494205580758204
21. Feature 193 (mi2slp02): 0.01066484872628446
22. Feature 98 (avesmk15): 0.01021966813955359
2

## Mutual Information

In [17]:
mi = mutual_info_regression(x_imputed, y)

In [18]:
# Get the indices of the features sorted by MI
sorted_indices = np.argsort(mi)[::-1]

# Print the feature rankings
for i, idx in enumerate(sorted_indices):
    print(f"{i+1}. Feature {idx} ({x_imputed.columns[idx]}): {mi[idx]}")

1. Feature 71 (neck20): 0.09842886747602364
2. Feature 286 (weight): 0.09727749780213113
3. Feature 287 (waist): 0.0873241827939939
4. Feature 289 (weight20): 0.07675859803569463
5. Feature 282 (bmi_s1): 0.07109587114484128
6. Feature 35 (gender): 0.05531517006422648
7. Feature 212 (loudsn02): 0.04834479317919893
8. Feature 217 (hostbr02): 0.0342579963337184
9. Feature 95 (yrsns15): 0.03382460482096761
10. Feature 47 (trig): 0.032975960762717804
11. Feature 211 (hosnr02): 0.030461151361249605
12. Feature 44 (hip): 0.030155287107155537
13. Feature 46 (hdl): 0.027772645226965764
14. Feature 99 (wine15): 0.02741246398130759
15. Feature 67 (syst220): 0.02518808039697884
16. Feature 288 (height): 0.024926957177759057
17. Feature 103 (asalw15): 0.024594529703248824
18. Feature 245 (pf_s1): 0.022759632640267924
19. Feature 51 (ankbp): 0.021664300729891117
20. Feature 1 (pptid): 0.0216333469633172
21. Feature 0 (nsrrid): 0.021261065822532288
22. Feature 244 (rawpf_s1): 0.01941652849043951
23. 